In [1]:
from dolfin import *
import meshio
%matplotlib inline

/opt/anaconda3/envs/fenicsproject/lib/python3.10/site-packages/dolfin/jit/jit.py:121: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  def compile_class(cpp_data, mpi_comm=MPI.comm_world):


In [2]:
import os
os.getcwd()

'/Users/abhigupta/Storage/codes/pd/adaptive-glaciers/2_codes/3_paper_problems/3-1-adaptive-vs-non-adaptive/mesh/adaptive'

In [3]:
import numpy as np

In [4]:
mesh = meshio.read("adaptive-glacier.med")
points, cell, cell_data, field_data = mesh.points, mesh.cells, mesh.cell_data, mesh.field_data

In [5]:
cell['tetra']

array([[2242, 2235, 2218, 2217],
       [1813,  499, 1828, 1830],
       [ 610,  378, 1739,  377],
       ...,
       [2124, 1123, 1024, 1122],
       [2124, 1123, 1122, 2190],
       [1210, 2192, 1209, 1128]])

In [6]:
a = cell['tetra']

In [7]:
a

array([[2242, 2235, 2218, 2217],
       [1813,  499, 1828, 1830],
       [ 610,  378, 1739,  377],
       ...,
       [2124, 1123, 1024, 1122],
       [2124, 1123, 1122, 2190],
       [1210, 2192, 1209, 1128]])

In [8]:
np.unique(mesh.point_data['point_tags'])

array([0])

In [9]:
cell[1]

KeyError: 1

In [10]:
points[:,:]

array([[242.5       , 750.        ,   0.        ],
       [242.5       , 750.        , 125.        ],
       [242.5       ,   0.        , 125.        ],
       ...,
       [400.24905755, 476.31073986,  76.95391171],
       [394.64933145, 281.27456913,  79.3087511 ],
       [451.77806228,  39.0085765 ,  79.24680385]])

In [11]:
meshio.write("mesh.xdmf", meshio.Mesh(
    points=points[:,:],
    cells={"tetra": a}))

In [46]:
meshio.write("line.xdmf", meshio.Mesh(
    points=points[:,[0,2]],
    cells={"line": cell["line"]},
    cell_data={"line": {"marker": -cell_data["line"]["cell_tags"]}}
))

In [47]:
meshio.write("point.xdmf", meshio.Mesh(
    points= points[:,[0,2]],
    cells = {"vertex": np.array([[i,] for i in range(len(points))])},# here the cell is a vertex and the cell num is vertex num
    cell_data={"vertex":{"marker": mesh.point_data["point_tags"]}}
))

In [48]:
mesh.point_data["point_tags"].max()

3

In [49]:
mesh.cell_tags

{-6: ['load_line'], -7: ['face']}

In [50]:
mesh.point_tags

{2: ['left_support'], 3: ['right_sipport']}

In [51]:
arr=cell_data["triangle"]["cell_tags"]
num=2

In [52]:
if num in arr:
    print(True)
else:
    print(False)

False


In [53]:
np.unique(arr)

array([-7])

# Demo

In [54]:
mesh_2d = Mesh()
with XDMFFile("mesh.xdmf") as infile:
    infile.read(mesh_2d)

In [55]:
mvc = MeshValueCollection("size_t", mesh_2d, 1)
with XDMFFile("line.xdmf") as infile:
    infile.read(mvc, "marker")

mf = cpp.mesh.MeshFunctionSizet(mesh_2d, mvc)

mf.array()[mf.array()>20]=0

with XDMFFile("mesh/mesh_function.xdmf") as xdmf_outfile:
    xdmf_outfile.write(mf)

In [56]:
mf.array().max()

6

In [57]:
mvc = MeshValueCollection("size_t", mesh_2d, 0)
with XDMFFile("point.xdmf") as infile:
    infile.read(mvc, "marker")

mf = cpp.mesh.MeshFunctionSizet(mesh_2d, mvc)

mf.array()[mf.array()>20]=0

with XDMFFile("mesh/point_mesh_function.xdmf") as xdmf_outfile:
    xdmf_outfile.write(mf)

In [58]:
mf.array().max()

3

In [59]:
mvc.dim()

0